<a href="https://colab.research.google.com/github/adrianacupp/NLP_with_Transformers/blob/main/Relation_Classification_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract dataset

In [1]:
#extract dataset directly from kaggle

In [2]:
# 1. Read the kaggle API token to interact with your kaggle account

from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"adracupp","key":"e74d1222705292f7c525d7e2a435b34f"}'}

In [3]:
#2. Series of commands to set-up for download

!ls -lha kaggle.json
!pip install -q kaggle # installing the kaggle package
!mkdir -p ~/.kaggle # creating .kaggle folder where the key should be placed
!cp kaggle.json ~/.kaggle/ # move the key to the folder
!pwd # checking the present working directory

-rw-r--r-- 1 root root 64 Aug  6 09:32 kaggle.json
/content


In [4]:
#3. giving rw access (if 401-nathorized)

!chmod 600 ~/.kaggle/kaggle.json

In [5]:
#4. Sanity check if able to access kaggle
!kaggle datasets list

ref                                                    title                                    size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------  --------------------------------------  -----  -------------------  -------------  ---------  ---------------  
waqi786/cats-dataset                                   🐾 Cats Dataset                            6KB  2024-07-31 06:12:49           1717         34  1.0              
priyamchoksi/credit-card-transactions-dataset          Credit Card Transactions Dataset        145MB  2024-07-23 00:31:47           1463         30  1.0              
abhayayare/employee-data-simulation-it-industry        Employee Data Simulation: IT Industry     4KB  2024-07-21 15:26:17           1941         33  1.0              
patricklford/global-ev-sales-2010-2024                 Global EV Sales: 2010-2024               83KB  2024-07-19 13:40:22           2539         35  1.0             

In [6]:
!kaggle datasets download -d daishinkan002/new-york-times-relation-extraction-dataset

Dataset URL: https://www.kaggle.com/datasets/daishinkan002/new-york-times-relation-extraction-dataset
License(s): unknown
 55% 5.00M/9.02M [00:00<00:00, 29.8MB/s]
100% 9.02M/9.02M [00:00<00:00, 50.5MB/s]


In [7]:
!pwd


/content


In [8]:
!ls -lha


total 9.1M
drwxr-xr-x 1 root root 4.0K Aug  6 09:32 .
drwxr-xr-x 1 root root 4.0K Aug  6 09:29 ..
drwxr-xr-x 4 root root 4.0K Aug  2 13:24 .config
-rw-r--r-- 1 root root   64 Aug  6 09:32 kaggle.json
-rw-r--r-- 1 root root 9.1M Jul 31  2021 new-york-times-relation-extraction-dataset.zip
drwxr-xr-x 1 root root 4.0K Aug  2 13:24 sample_data


In [9]:
!unzip new-york-times-relation-extraction-dataset.zip


Archive:  new-york-times-relation-extraction-dataset.zip
  inflating: dataset/test.json       
  inflating: dataset/train.json      
  inflating: dataset/valid.json      


In [10]:
# Function to extract relations from a JSON object
def extract_relations(json_line):
    relations = []
    sent_text = json_line['sentText']
    for relation in json_line['relationMentions']:
        em1_text = relation['em1Text']
        em2_text = relation['em2Text']
        label = relation['label']
        relations.append((sent_text, em1_text, em2_text, label))
    return relations

In [11]:
import json

# Define the file path
input_file = 'dataset/train.json'

# Initialize a list to hold the data
data = []

# Initialize a counter for the number of lines processed
count = 0

# Read the JSON file
with open(input_file, 'r') as file:
    for line in file:
        json_line = json.loads(line)  # Load JSON object from the line
        relations = extract_relations(json_line)  # Extract relations
        data.extend(relations)  # Add extracted relations to the data list
        count += 1  # Increment the counter

In [12]:
# Print the number of lines processed and the first few extracted relations
print(f"Processed {count} lines.")
print(f"First few extracted relations: {data[:5]}")
# JSON object can contain multiple relation mentions --> df entries may be more

Processed 56196 lines.
First few extracted relations: [('Massachusetts ASTON MAGNA Great Barrington ; also at Bard College , Annandale-on-Hudson , N.Y. , July 1-Aug .', 'Annandale-on-Hudson', 'Bard College', '/location/location/contains'), ('North Carolina EASTERN MUSIC FESTIVAL Greensboro , June 25-July 30 .', 'North Carolina', 'Greensboro', '/location/location/contains'), ("It will be the final movie credited to Debra Hill , a film producer and native of Haddonfield , who produced '' Halloween '' and was considered a pioneering woman in film .", 'Debra Hill', 'Haddonfield', '/people/person/place_of_birth'), ("In a 3-0 victory over the Crew on Saturday in Columbus , Ohio , goalkeeper Zach Wells stopped Kyle Martino 's penalty kick , only the third unsuccessful penalty in 20 attempts in M.L.S. this season .", 'Ohio', 'Columbus', '/location/location/contains'), ("The United States ambassador to Mexico , Tony Garza , said in a statement that he had directed the American Consulate in Nuev

In [13]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

# Convert the extracted data to a pandas DataFrame
df = pd.DataFrame(data, columns=['sentence', 'entity1', 'entity2', 'label'])

output_path = '/content/drive/MyDrive/Colab Notebooks/extracted_relations.csv'

df.to_csv(output_path, index=False)

Mounted at /content/drive


In [14]:
df.head()

,sentence,entity1,entity2,label
0,Massachusetts ASTON MAGNA Great Barrington ; a...,Annandale-on-Hudson,Bard College,/location/location/contains
1,North Carolina EASTERN MUSIC FESTIVAL Greensbo...,North Carolina,Greensboro,/location/location/contains
2,It will be the final movie credited to Debra H...,Debra Hill,Haddonfield,/people/person/place_of_birth
3,In a 3-0 victory over the Crew on Saturday in ...,Ohio,Columbus,/location/location/contains
4,"The United States ambassador to Mexico , Tony ...",Mexico,Nuevo Laredo,/location/location/contains


#EDA

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94222 entries, 0 to 94221
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  94222 non-null  object
 1   entity1   94222 non-null  object
 2   entity2   94222 non-null  object
 3   label     94222 non-null  object
dtypes: object(4)
memory usage: 2.9+ MB


In [16]:
df['label'].value_counts()

label
/location/location/contains                           45608
/people/person/nationality                             7198
/location/country/capital                              6718
/people/person/place_lived                             6362
/location/administrative_division/country              5658
/location/country/administrative_divisions             5658
/business/person/company                               4950
/location/neighborhood/neighborhood_of                 4941
/people/person/place_of_birth                          2813
/people/deceased_person/place_of_death                 1752
/business/company/founders                              712
/people/person/children                                 441
/business/company/place_founded                         360
/business/company_shareholder/major_shareholder_of      249
/business/company/major_shareholders                    249
/sports/sports_team_location/teams                      186
/sports/sports_team/location      

In [17]:
df[df['label'].str.contains("business")]['label'].value_counts()

label
/business/person/company                              4950
/business/company/founders                             712
/business/company/place_founded                        360
/business/company_shareholder/major_shareholder_of     249
/business/company/major_shareholders                   249
/business/company/advisors                              39
/business/company/industry                               1
Name: count, dtype: int64

In [18]:
#Change the first letter to an empty string in the label column
df['label'] = df['label'].str[1:]
df.head()

,sentence,entity1,entity2,label
0,Massachusetts ASTON MAGNA Great Barrington ; a...,Annandale-on-Hudson,Bard College,location/location/contains
1,North Carolina EASTERN MUSIC FESTIVAL Greensbo...,North Carolina,Greensboro,location/location/contains
2,It will be the final movie credited to Debra H...,Debra Hill,Haddonfield,people/person/place_of_birth
3,In a 3-0 victory over the Crew on Saturday in ...,Ohio,Columbus,location/location/contains
4,"The United States ambassador to Mexico , Tony ...",Mexico,Nuevo Laredo,location/location/contains


In [19]:
# Change all the forward slashes to underscores in the label column
df['label'] = df['label'].str.replace('/', '_')
df

,sentence,entity1,entity2,label
0,Massachusetts ASTON MAGNA Great Barrington ; a...,Annandale-on-Hudson,Bard College,location_location_contains
1,North Carolina EASTERN MUSIC FESTIVAL Greensbo...,North Carolina,Greensboro,location_location_contains
2,It will be the final movie credited to Debra H...,Debra Hill,Haddonfield,people_person_place_of_birth
3,In a 3-0 victory over the Crew on Saturday in ...,Ohio,Columbus,location_location_contains
4,"The United States ambassador to Mexico , Tony ...",Mexico,Nuevo Laredo,location_location_contains
...,...,...,...,...
94217,Here we have a 172-acre island with four-star ...,Manhattan,Ellis Island,location_location_contains
94218,One was for St. Francis de Sales Roman Catholi...,Belle Harbor,Queens,location_neighborhood_neighborhood_of
94219,One was for St. Francis de Sales Roman Catholi...,Queens,Rosedale,location_location_contains
94220,One was for St. Francis de Sales Roman Catholi...,Rosedale,Queens,location_neighborhood_neighborhood_of


In [20]:
# Print the unique labels in the label column
unique_labels = df['label'].unique()
sorted(unique_labels)

['business_company_advisors',
 'business_company_founders',
 'business_company_industry',
 'business_company_major_shareholders',
 'business_company_place_founded',
 'business_company_shareholder_major_shareholder_of',
 'business_person_company',
 'location_administrative_division_country',
 'location_country_administrative_divisions',
 'location_country_capital',
 'location_location_contains',
 'location_neighborhood_neighborhood_of',
 'people_deceased_person_place_of_death',
 'people_ethnicity_geographic_distribution',
 'people_ethnicity_people',
 'people_person_children',
 'people_person_ethnicity',
 'people_person_nationality',
 'people_person_place_lived',
 'people_person_place_of_birth',
 'people_person_profession',
 'people_person_religion',
 'sports_sports_team_location',
 'sports_sports_team_location_teams']

# Load the Model

In [21]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split

In [22]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [23]:
# 1. Preprocess the data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
label_encoder = {label: i for i, label in enumerate(df['label'].unique())}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [24]:
def preprocess_data(row):
    sentence = row['sentence']
    # Tokenize sentence
    tokens = tokenizer.tokenize(sentence)

    # Encode entities positions
    encoded_sentence = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    ).to(device)

    return {
        'input_ids': encoded_sentence['input_ids'].squeeze(),
        'attention_mask': encoded_sentence['attention_mask'].squeeze(),
        'label': torch.tensor(label_encoder[row['label']])
    }

In [25]:
train_data = train_df.apply(preprocess_data, axis=1).tolist()
val_data = val_df.apply(preprocess_data, axis=1).tolist()

In [26]:
# 2. Create custom dataset class
class RelationshipDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Create dataset and dataloader
train_dataset = RelationshipDataset(train_data)
val_dataset = RelationshipDataset(val_data)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [27]:
class RelationshipExtractionModel(torch.nn.Module):
    def __init__(self, num_labels):
        super().__init__()
        self.bert = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        return logits


In [28]:
# 4. Set up training loop
num_labels = len(df['label'].unique())
model = RelationshipExtractionModel(num_labels)
optimizer = AdamW(model.parameters(), lr=2e-5)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [29]:
num_epochs = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

RelationshipExtractionModel(
  (bert): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, o

In [30]:
from sklearn.metrics import accuracy_score
def evaluate(model, dataloader):
    model.eval()
    total_preds = []
    total_labels = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            _, preds = torch.max(outputs, dim=1)

            total_preds.extend(preds.cpu().tolist())
            total_labels.extend(labels.cpu().tolist())

    return accuracy_score(total_labels, total_preds)

In [31]:
from tqdm.auto import tqdm

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix({'loss': total_loss / (progress_bar.n + 1)})

    # Calculate validation accuracy
    val_accuracy = evaluate(model, val_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs} completed. Validation Accuracy: {val_accuracy:.4f}")

Epoch 1/5:   0%|          | 0/4712 [00:00<?, ?it/s]

Epoch 1/5 completed. Validation Accuracy: 0.6598


Epoch 2/5:   0%|          | 0/4712 [00:00<?, ?it/s]

Epoch 2/5 completed. Validation Accuracy: 0.6565


Epoch 3/5:   0%|          | 0/4712 [00:00<?, ?it/s]

Epoch 3/5 completed. Validation Accuracy: 0.6489


Epoch 4/5:   0%|          | 0/4712 [00:00<?, ?it/s]

Epoch 4/5 completed. Validation Accuracy: 0.6520


Epoch 5/5:   0%|          | 0/4712 [00:00<?, ?it/s]

Epoch 5/5 completed. Validation Accuracy: 0.6517


In [32]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def predict_relationship(sentence):
    # Tokenize sentence
    tokens = tokenizer.tokenize(sentence)

    # Encode entities positions
    encoded_sentence = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoded_sentence['input_ids'].to(device)
    attention_mask = encoded_sentence['attention_mask'].to(device)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        _, preds = torch.max(outputs, dim=1)

    # Convert prediction back to label
    reverse_label_encoder = {v: k for k, v in label_encoder.items()}
    predicted_label = reverse_label_encoder[preds.item()]

    return predicted_label

In [33]:
sentence = "IBM is an American multinational technology company headquartered in Armonk"

predicted_relationship = predict_relationship(sentence)
print(f"Predicted relationship: {predicted_relationship}")

Predicted relationship: business_company_place_founded
